In [1]:
import pandas as pd

In [78]:
df = pd.read_csv('./actorfilms.csv').drop_duplicates(subset=['FilmID'])
df = df.reset_index(drop=True)

df

,Actor,ActorID,Film,Year,Votes,Rating,FilmID
0,Fred Astaire,nm0000001,Ghost Story,1981,7731,6.3,tt0082449
1,Fred Astaire,nm0000001,The Purple Taxi,1977,533,6.6,tt0076851
2,Fred Astaire,nm0000001,The Amazing Dobermans,1976,369,5.3,tt0074130
3,Fred Astaire,nm0000001,The Towering Inferno,1974,39888,7.0,tt0072308
4,Fred Astaire,nm0000001,Midas Run,1969,123,4.8,tt0064664
...,...,...,...,...,...,...,...
44451,Yeo-bin Jeon,nm9256310,The Running Actress,2017,102,5.5,tt7341842
44452,Yeo-bin Jeon,nm9256310,The Treacherous,2015,1195,6.1,tt4844288
44453,Janhvi Kapoor,nm9427099,Gunjan Saxena: The Kargil Girl,2020,24696,5.3,tt10350626
44454,Janhvi Kapoor,nm9427099,Ghost Stories,2020,4485,4.3,tt10230404


In [94]:
import pandas as pd
import concurrent.futures
from imdb import IMDb


df_cast = pd.read_csv('./actorfilms.csv').drop_duplicates(subset=['FilmID']).reset_index(drop=True)


new_df = df_cast[['FilmID', 'Year']].head(4)
unique_df = new_df.drop_duplicates(subset=['FilmID'])

list_ = unique_df.copy(deep=True)

list_ = unique_df
# film_ids = unique_df['FilmID'].tolist()
print(list_)


      FilmID  Year
0  tt0082449  1981
1  tt0076851  1977
2  tt0074130  1976
3  tt0072308  1974


In [86]:
# Create a new None column and set all values to None initially
unique_df = unique_df.assign(Genre=None)
unique_df = unique_df.assign(Cast=None)
unique_df = unique_df.assign(Director=None)
unique_df = unique_df.assign(Year=None)

In [93]:

ia = IMDb()

# Define a function to get data from IMDb
def get_imdb_data(film_id):
    rows_with_film_id = df[df['FilmID'] == film_id]


    # # Print the index
    print(f"index is {rows_with_film_id.index[0]} of 44455")
    print('\n')
    film_id = film_id.lstrip('tt')
    print(f'film_id is {film_id}, Starting Download Data ...')
    try:
        movie = ia.get_movie(film_id)
        genre = movie['genres'][0] if 'genres' in movie else None
        cast = [actor['name'] for actor in movie['cast']][:10] if 'cast' in movie else None
        director = movie['director'][0]['name'] if 'director' in movie else None
        year = movie['year']
        print(f'Movie {movie}')
        print('Download completed')
        return genre, cast, director, year
    except Exception as e:
        print(f"Error getting data for film {film_id}: {e}")
        return None, None, None, None

# Create a list of film IDs to process
film_ids = list_['FilmID'].tolist()

# Define the number of worker threads to use
num_threads = 4

# Create a thread pool executor with the specified number of threads
with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Submit the get_imdb_data function for each film ID to the executor
    futures = [executor.submit(get_imdb_data, film_id) for film_id in film_ids]

    # Wait for all of the futures to complete
    results = concurrent.futures.wait(futures)

print('\nAssign Data to imdb_data')
# Extract the results from the completed futures and add them to the DataFrame
for i, result in enumerate(results[0]):
    genre, cast, director, year = result.result()
    if genre is not None:
        unique_df.iloc[i, unique_df.columns.get_loc('Genre')] = genre
    if cast is not None:
        if isinstance(cast, (list, tuple)):
            cast = ', '.join(cast)
        unique_df.iloc[i, unique_df.columns.get_loc('Cast')] = cast
    if director is not None:
        unique_df.iloc[i, unique_df.columns.get_loc('Director')] = director
    if year is not None:
        unique_df.iloc[i, unique_df.columns.get_loc('Year')] = year


# Save the updated DataFrame to a new CSV file
print('\nCreate imdb_data')
unique_df.to_csv('imdb_data.csv', index=False)
df_imdb_data = pd.read_csv('./imdb_data.csv')
print('completed')

index is 0 of 44455


film_id is 0082449, Starting Download Data ...
index is 1 of 44455


film_id is 0076851, Starting Download Data ...
index is 2 of 44455


film_id is 0074130, Starting Download Data ...
index is 3 of 44455


film_id is 0072308, Starting Download Data ...
Movie The Purple Taxi
Download completed
Movie The Amazing Dobermans
Download completed
Movie Ghost Story
Download completed


2023-07-20 19:04:29,400 CRITICAL [imdbpy] /home/selector/.local/lib/python3.10/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0072308/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "/home/selector/.local/lib/python3.10/site-packages/imdb/parser/http/__init__.py", line 234, in retrieve_unicode
    content = response.read()
  File "/usr/lib/python3.10/http/client.py", line 459, in read
    return self._read_chunked(amt)
  File "/usr/lib/python3.10/http/client.py", line 582, in _read_chunked
    chunk_left = self._get_chunk_left()
  File "/usr/lib/python3.10/http/client.py", line 565, in _get_chunk_left
    chunk_left = self._read_next_chunk_size()
  File "/usr/lib/python3.10/http/client.py", line 525, in _read_next_chunk_size
    line = self.fp.readline(

Error getting data for film 0072308: {'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0072308/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')}

Assign Data to imdb_data

Create imdb_data
completed
